# 활성화 함수

* `__init__()` 함수에서는 모델에서 사용될 모듈(nn.Linear 등)과 activation function (활성화 함수) 등을 정의함
* forward() 함수에서 실행되어야 하는 연산에 활성화 함수도 적용하면 됨
* 주요 활성화 함수

  * 시그모이드 함수 : nn.Sigmoid()
  * ReLU 함수 : nn.ReLU()
  * Leaky ReLU 함수 : nn.LeakyReLU()

In [4]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()  # 부모 클래스의 생성자 호출
    # 명시적으로 인자명을 써주는 경우도 많이 쓰임
    self.linear = nn.Linear(in_features=input_dim, out_features=output_dim)
    # self.activation = nn.Sigmoid() # 시그모이드 활성화 함수 추가
    self.activation = nn.ReLU()  # ReLU 활성화 함수로 변경

  def forward(self, x):
    return self.activation(self.linear(x))  # 선형 변환 후 시그모이드 활성화 함수 적용

In [5]:
x = torch.ones(4) # 입력 데이터
y = torch.zeros(3) # 타겟 데이터
model = LinearRegressionModel(4, 3)
loss_function = nn.MSELoss()

In [6]:
learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


for epoch in range(nb_epochs + 1):
  
  y_pred = model(x)  # 모델에 입력 데이터 전달
  loss = loss_function(y_pred, y)  # 손실 계산

  optimizer.zero_grad()  # 기울기 초기화
  loss.backward()  # 역전파 수행
  optimizer.step()  # 가중치 업데이트

In [7]:
print(loss)
for param in model.parameters():
    print(param)

tensor(2.3981e-14, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.3357, -0.0423, -0.0352,  0.2639],
        [ 0.1099, -0.1510,  0.0899, -0.2836],
        [ 0.3906,  0.1287, -0.4643, -0.0335]], requires_grad=True)
Parameter containing:
tensor([ 0.1494, -0.1243, -0.2704], requires_grad=True)


### 다층 레이어 구현

* input layer -> hidden layer -> output layer 순으로 순차적으로 작성해주면 됨

  * 내부 행렬곱 조건만 유의해주면 됨

* activation function 적용은 output layer에는 적용하지 않는 것이 일반적임

In [8]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):

  def __init__(self, input_dim, output_dim):
    super().__init__()  # 부모 클래스의 생성자 호출
    self.linear1 = nn.Linear(input_dim, 10)
    self.linear2 = nn.Linear(10, 10)
    self.linear3 = nn.Linear(10, 10)
    self.linear4 = nn.Linear(10, output_dim)
    self.activation = nn.ReLU(0.1)  # ReLU 활성화 함수 사용

  def forward(self, x):
    # |x| = (input_dim, output_dim)
    hidden = self.activation(self.linear1(x)) # |hidden| = (input_dim, 5)
    hidden = self.activation(self.linear2(hidden)) # |hidden| = (5, 5)
    hidden = self.activation(self.linear3(hidden)) # |hidden| = (5, 5)
    y = self.linear4(hidden) # 마지막 출력에는 activation 함수 사용하지 않는 것이 일반적
    return y

In [9]:
x = torch.ones(4) # 입력 데이터
y = torch.zeros(3) # 타겟 데이터
model = LinearRegressionModel(4, 3)
loss_function = nn.MSELoss()

In [10]:
learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
  y_pred = model(x)  # 모델에 입력 데이터 전달
  loss = loss_function(y_pred, y)  # 손실 계산

  optimizer.zero_grad()  # 기울기 초기화
  loss.backward()  # 역전파 수행
  optimizer.step()  # 가중치 업데이트

In [11]:
print(loss)
for param in model.parameters():
    print(param)

tensor(2.6077e-12, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.1204,  0.4659,  0.1615, -0.4481],
        [-0.0839,  0.1950, -0.3970,  0.2374],
        [-0.0385, -0.3775, -0.4988,  0.0946],
        [-0.4180, -0.1393, -0.4911,  0.3324],
        [-0.2099, -0.3720, -0.0397, -0.2565],
        [-0.2442, -0.4150, -0.1952, -0.2070],
        [ 0.1258, -0.1804,  0.0869, -0.3465],
        [ 0.3508, -0.0824,  0.1861,  0.1149],
        [-0.1007,  0.4691, -0.1621, -0.1505],
        [-0.1919, -0.2876, -0.0688,  0.1972]], requires_grad=True)
Parameter containing:
tensor([ 0.4891, -0.0945, -0.1008,  0.4157, -0.2431, -0.0073, -0.1456, -0.4737,
        -0.1212, -0.4458], requires_grad=True)
Parameter containing:
tensor([[-0.1044, -0.2551,  0.1321, -0.0971, -0.2382,  0.2262, -0.0278, -0.0882,
         -0.1404, -0.1707],
        [-0.0635,  0.2739,  0.0013, -0.0202,  0.1146,  0.1113, -0.2956,  0.1518,
         -0.0734,  0.2853],
        [-0.0194, -0.1272, -0.0691, -0.0453,  0.1260, -0.2701

### nn.Sequential

* nn.Sequential은 순서를 갖는 모듈의 컨테이너를 의미함
* 순차적으로 연산되는 레이어만 있을 경우에는, nn.Sequential 을 통해 순서대로 각 레이어를 작성하면 그대로 실행됨

  * 중간에 activation function이 적용된다면, activation function도 순서에 맞게 넣어주면 자동 계산 됨

In [14]:
input_dim = x.size(0)
output_dim = y.size(0)

model = nn.Sequential(
    nn.Linear(input_dim, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, output_dim)
)

In [19]:
loss_function = nn.MSELoss()
learning_rate = 0.01
nb_epochs = 1000

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
  y_pred = model(x)  # 모델에 입력 데이터 전달
  loss = loss_function(y_pred, y)  # 손실 계산

  optimizer.zero_grad()  # 기울기 초기화
  loss.backward()  # 역전파 수행
  optimizer.step()  # 가중치 업데이트

In [18]:
print(loss)
for param in model.parameters():
    print(param)

tensor(1.0070e-13, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.4381,  0.3992,  0.2607, -0.2762],
        [-0.1656, -0.2880,  0.0890, -0.2153],
        [ 0.4674,  0.3049, -0.1792, -0.0701],
        [ 0.1322,  0.0445,  0.3819,  0.2812],
        [ 0.1160,  0.3883,  0.1645, -0.4500],
        [-0.4363,  0.0233,  0.2574,  0.3525],
        [ 0.4328, -0.4439,  0.1300, -0.2645],
        [-0.4831, -0.4115,  0.2795, -0.4369],
        [-0.1175, -0.4448, -0.3968, -0.3317],
        [-0.0493,  0.2065,  0.3251,  0.4387]], requires_grad=True)
Parameter containing:
tensor([ 0.2167,  0.4244, -0.1690,  0.3955,  0.2108,  0.3322,  0.2178, -0.3951,
        -0.1961, -0.2803], requires_grad=True)
Parameter containing:
tensor([[-0.0328, -0.1617, -0.0312,  0.2393,  0.0604,  0.2848,  0.1242,  0.1320,
          0.2879,  0.0743],
        [ 0.2898,  0.0837,  0.2301, -0.2395, -0.0403,  0.1466,  0.0244,  0.1484,
         -0.2794, -0.3033],
        [ 0.0583,  0.2014, -0.0573, -0.1114, -0.3068, -0.0056